In [1]:
!nvidia-smi

Fri Mar 21 17:27:20 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.05              Driver Version: 560.35.05      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla V100-PCIE-16GB           Off |   00000000:18:00.0 Off |                    0 |
| N/A   43C    P0             40W /  250W |       1MiB /  16384MiB |      1%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import os
import sys
sys.path.append('..')  # TODO: configure this at apptainer level
from libs.utils.logger import get_logger, setup_loggers

user = os.getenv('USER')
%cd /home/{user}/Unlearn-Saliency/PEM_composition_img_gen

logger = get_logger('main')
setup_loggers(modules_info=['libs.'])

from libs.utils.training import launch_accelerated_training
from libs.unlearner import UnlearnerLora


launch_accelerated_training(unlearner=UnlearnerLora(
    pretrained_model_name_or_path="stable-diffusion-v1-5/stable-diffusion-v1-5",
    dataset_forget_name="./assets/imagenette_splits/n02979186/train_forget",
    dataset_retain_name="./assets/imagenette_splits/n02979186/train_retain",
    dataloader_num_workers=2,
    resolution=512,
    mixed_precision="fp16",
    train_batch_size=4,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    max_grad_norm=1.0,
    lr_scheduler="cosine",
    lr_warmup_steps=0,
    output_dir=f"./assets/lora/imagenette_splits/n02979186",
    num_train_epochs=1,
    validation_epochs=1,
    checkpointing_steps=500,
    validation_prompt="Picture of a church",
    num_validation_images=1,
    seed=42
))

/home/benle1/Unlearn-Saliency/PEM_composition_img_gen


Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


[2025-03-21 17:27:39 +0100] libs.trainer INFO     Distributed environment: DistributedType.NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: fp16



{'sample_max_value', 'thresholding', 'clip_sample_range', 'variance_type', 'dynamic_thresholding_ratio', 'timestep_spacing', 'prediction_type', 'rescale_betas_zero_snr'} was not found in config. Values will be initialized to default values.
{'use_quant_conv', 'latents_std', 'scaling_factor', 'shift_factor', 'use_post_quant_conv', 'force_upcast', 'mid_block_add_attention', 'latents_mean'} was not found in config. Values will be initialized to default values.
{'addition_embed_type_num_heads', 'mid_block_type', 'conv_in_kernel', 'dual_cross_attention', 'dropout', 'reverse_transformer_layers_per_block', 'time_embedding_type', 'upcast_attention', 'num_attention_heads', 'projection_class_embeddings_input_dim', 'only_cross_attention', 'class_embed_type', 'conv_out_kernel', 'transformer_layers_per_block', 'mid_block_only_cross_attention', 'use_linear_projection', 'num_class_embeds', 'cross_attention_norm', 'resnet_time_scale_shift', 'attention_type', 'addition_time_embed_dim', 'timestep_post_a

Resolving data files:   0%|          | 0/895 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/7632 [00:00<?, ?it/s]

[2025-03-21 17:27:52 +0100] libs.trainer INFO     ***** Running training *****
[2025-03-21 17:27:52 +0100] libs.trainer INFO       Num examples = 894 + 7631
[2025-03-21 17:27:52 +0100] libs.trainer INFO       Num Epochs = 1
[2025-03-21 17:27:52 +0100] libs.trainer INFO       Instantaneous batch size per device = 4
[2025-03-21 17:27:52 +0100] libs.trainer INFO       Total train batch size (w. parallel, distributed & accumulation) = 32
[2025-03-21 17:27:52 +0100] libs.trainer INFO       Gradient Accumulation steps = 8
[2025-03-21 17:27:52 +0100] libs.trainer INFO       Total optimization steps = 28


Steps:   0%|          | 0/28 [00:00<?, ?it/s]